In [1]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [ ]:
# pd.set_option("display.max_rows", 300)
pd.set_option('display.max_colwidth', None)

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

In [ ]:
terms_header_list = ["AACR2_FLAG", "DISPLAY_DATE", "DISPLAY_NAME", "DISPLAY_ORDER", "END_DATE", "HISTORIC_FLAG", "OTHER_FLAGS", "PREFERRED", "START_DATE", "SUBJECT_ID", "TERM", "TERM_ID", "VERNACULAR"]
terms_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str}
lang_rels_header_list = ["LANGUAGE_CODE", "PREFERRED", "SUBJECT_ID", "TERM_ID", "QUALIFIER", "TERM_TYPE", "PART_OF_SPEECH", "LANG_STAT"]
lang_rel_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str, 'LANGUAGE_CODE': str}
spreadsheet = ['identifier', 'media', 'subject', 'title', 'subjects', 'join_concept']
narrow_list = ['identifier', 'media', 'subject', 'title', 'subjects', 'SUBJECT_ID_x', 'TERM_ID', 'join_concept']
concept_list = ['PREFERRED_x', 'SUBJECT_ID_x', 'TERM', 'TERM_ID', 'LANGUAGE_CODE', 'QUALIFIER', 'LANG_STAT', 'use_concept', 'join_concept']
alt_lang_list = ['identifier', 'media', 'subjects', 'SUBJECT_ID_x', 'TERM_ID_x', 'use_concept_x', 'LANGUAGE_CODE_y', 'PREFERRED_y_y', 'SUBJECT_ID_y_y', 'TERM_ID_y', 'use_concept_y']

In [ ]:
def read_aat_terms(file_path: str, names: list, data_types: dict) -> pd.DataFrame:
    data = pd.read_csv(file_path
                       , sep = '\t'
                       , warn_bad_lines=True
                       , error_bad_lines=False
                       , names=names
                       , dtype=data_types)
    return data

In [ ]:
def format_column_names(df: pd.DataFrame) -> pd.DataFrame:
    column_names = list(map(lambda x: x.strip().lower().replace("dcterms:", ""), df.columns))
    df = df.set_axis(column_names, axis=1, inplace=False)
    return df

In [ ]:
def get_unique_subjects(box_number: int=1) -> list:
    subjects = []
    single_list = []
    box_id = f'Box {box_number}'
    try:
        df = s.find(box_id).to_frame()
        df = format_column_names(df)
        subjects = df['subject'].unique().tolist()
        for term in subjects:
            split_terms = str(term).split(',')
            for ind_term in split_terms:
                single_list.append(ind_term.strip())
    except:
        print(f'ParserError: could not access {box_id}')

    return single_list

In [ ]:
def combine_boxes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """assuming df1 and df2 are dataframes with same schema, combines them and returns them"""
    df_concat = pd.concat([df1, df2])
    return df_concat

In [ ]:
def split_df(df: pd.DataFrame) -> pd.DataFrame:
    """given a dataframe with multiple concepts specified, splits them and returns them 
    as one row per concept
    """
    split_df = df['subject'].str.split(',', expand=True).apply(Series, 1).stack()
    split_df.index = split_df.index.droplevel(-1)
    split_df.name = 'subjects'
#     del df['subjects']
    x = df.join(split_df)
    x['subjects'] = x['subjects'].str.strip()
    x['join_concept'] = x['subjects'].str.lower()
    
#   put back the comma back where it was replaced with ∆ in the check_for_commas function
    x['subject'] = x['subject'].str.replace('∆', ',')
    x['subjects'] = x['subjects'].str.replace('∆', ',')
    x['join_concept'] = x['join_concept'].str.replace('∆', ',')
    
    return x

In [ ]:
def load_metadata() -> pd.DataFrame:
    """load the metadata that will be used
    files were downloaded from: http://aatdownloads.getty.edu/
    """
    terms = read_aat_terms('./AAT_Files/TERM.out', terms_header_list, terms_dtype)
    lang_rels = read_aat_terms('./AAT_Files/LANGUAGE_RELS.out', lang_rels_header_list, lang_rel_dtype)
    concepts = (pd.merge(terms, lang_rels, how = 'inner', on='TERM_ID'))
    concepts['full_concept'] = (concepts.TERM + ' (' + concepts.QUALIFIER + ')')
    concepts['use_concept'] = (concepts.full_concept.fillna(concepts.TERM))
    concepts['join_concept'] = (concepts.use_concept.str.lower())

    return concepts

In [ ]:
def return_box(box_num: int) -> pd.DataFrame:
    box_id = f'Box {str(box_num)}'
    df = s.find(box_id).to_frame()
    df = format_column_names(df)
    return df

In [ ]:
def check_for_commas(df: pd.DataFrame, box_number: int) -> pd.DataFrame:
    '''
    1. create a dictionary from the media and subjects for the indexes
    2. cross reference the subjects of each box (df) to see if they contain any of the concepts that contain a comma (concepts_comma)
    3. replace the internal comma of any of those concepts with a '∆'
    4. replace the full concept for parsing by comma
    5. later, during the parsing, replace any internal commas (now '∆') with a comma 
    '''
    
    comma_change_file = './log_files/comma_change_file_log.csv'
    
#     create a dictionary from the media and subject for the box (df)
    df_media_subject = {}
    df_media_subject = df.set_index('media').to_dict()['subject']
    
#   iterate through the df for each record in the concepts_comma list and replace ',' with ∆
    log_list = []
    for i, media in enumerate(df_media_subject):
        clear_output(wait=True)
        print(f'working on box: {box_number}, row {i+1}')

        for j, conc in enumerate(concepts_comma):
            clear_output(wait=True)
            print(f'working on box: {box_number}, row {i+1}, concept: {j+1}')
            found_conc = False
            comma_conc = conc+','
        
            try:
                search_within = df_media_subject[media]
                re_conc = conc.replace('(', '\(')
                re_conc = re_conc.replace(')', '\)')

                if comma_conc in search_within:
                    found_conc = True
                if bool(re.search(f'{re_conc}$', search_within)):
                    found_conc = True

                if found_conc:
                    now = datetime.now()
                    current_time = now.strftime("%m-%d-%y %H:%M:%S")

                    log_list.append([f'{current_time}: {i}: {j}: {media} contains subject: {conc}'])
        #                 print(f'{i}: {j}: {media} contains subject: {conc}')
                    df_replace = df.loc[df['media'] == media]
                    try:
                        x = df_replace.to_numpy()[0][3]
                        x = x.replace(conc, conc.replace(',', '∆'))
                        df.loc[df['media'] == media, 'subject'] = x
                    except IndexError:
                        print(f'Index Error, media = {media}, subject = {x}')
                        break
            except TypeError:
                continue
        
    with open(comma_change_file, 'a', newline= '') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerows(log_list)
    
    return(df)

## Load the spreadsheet information

In [ ]:
# if HttpAccessTokenRefreshError, close everything and start over

sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

from the spreadsheet information, identify the sheets that start with 'Box' to define the boxes to be read into the dataframe

In [ ]:
box_list = []
box_list_sheets = s.sheets.titles()
# box_list_sheets = ['Box 2','Box 3','Box 4','Box 5 ','Box 6','Box 7','Box 8','Box 9','Box 10','Box 11','Box 12','Box 13','Box 14','Box 19','Box 20','Box 21','Box 26','Box 27','Box 28','Box 35','Box 36','Box 37','Box 38','Box 39','Box 40','Box 41','Box 43','Box 44','Box 45','Box 46','Box 47','Box 48','Box 49','Box 50','Box 51','Box 52','Box 53','Box 54','Box 55','Box 56','Box 58','Box 59','Box 60','Box 61','Box 62','Box 63','Box 64','Box 65','Box 66','Box 67','Box 68','Box 69']

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

## Load the Metadata from Getty AAT

In [ ]:
# get the metatadata
concepts = load_metadata()

In [ ]:
# review the metadata for concepts containing a comma and return those as a list
concepts_drop = concepts[concepts['use_concept'].notna()]
concepts_search = concepts_drop[concepts_drop['use_concept'].str.contains(',')]
concepts_comma = concepts_search['use_concept'].to_list()

## Read the data from the spreadsheet into the local dataframe

In [ ]:
for i, box_number in enumerate(box_list):
    df = return_box(box_number)
    clear_output(wait=True)
    print(f"working on Box {box_number}")
    df = check_for_commas(df, box_number)
    try:
        df = split_df(df)
    except:
        print(f'failed to split df for box number {box_number}')
    if i+1 == 1:
        df_combined = df
    else:
        df_combined = combine_boxes(df_combined, df)
clear_output(wait=True)    
print(f'complete, imported {i+1} boxes')

In [ ]:
# get some basic metrics about the imported data

items_with_subjects = len(df_combined[df_combined['subject'].notnull()])
items_without_subjects = len(df_combined[df_combined['subject'].isnull()])
total_items = len(df_combined)
print(f'total items: {total_items}\nitems with subjects {items_with_subjects}\nitems without subjects {items_without_subjects}')
unique_subjects = len(df_combined['subjects'].unique())
print(f'there are {unique_subjects} unique subjects found')

In [ ]:
# get the boxid
df_combined['boxid_temp'] = df_combined['media'].str.extract(r"(_0\d+_)")
df_combined = df_combined[df_combined['media'].notna()]
df_combined['boxid_temp'] = df_combined['boxid_temp'].str.replace("_","")
df_combined['boxid_temp'] = pd.to_numeric(df_combined['boxid_temp'], errors='coerce')
df_combined = df_combined.dropna(subset=['boxid_temp'])
df_combined['boxid_temp'] = df_combined['boxid_temp'].astype('int')
df_combined['boxid_temp'] = df_combined['boxid_temp'].astype('str')
df_combined['BoxId'] = 'Box '+df_combined['boxid_temp']
df_combined.drop('boxid_temp', inplace=True, axis=1)

## Add in Tab Color

The colors of each tab are identified in 'get_spreadsheets_color.ipynb'. That script creates a CSV file and that is read in and added to the df_combined object

Read in the color match file created here:
http://localhost:8888/notebooks/get_spreadsheets.ipynb

use that to create the list of titles based on color of the tab

In [ ]:
color_match_df = pd.read_csv('csv_files/color_match', sep='\t')
# color_match_df

In [ ]:
df_combined_color = pd.merge(df_combined, color_match_df, on='BoxId')

## Analyze the Subject Matches

The df_combined_color is the dataframe of parsed metadata from the spreadsheet
The concepts object is the dataframe of parsed metadata from AAT

The resulting merged dataframe is based on how they match against the "join_concept" or parsed indivdual subject being found in the AAT list

In [ ]:
subjects_concat_merge = pd.merge(df_combined_color, concepts, how="left", on='join_concept')

In [ ]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
subjects_concat_merge[narrow_list]

## Display the Subjects that Aren't Matching an AAT Concept/Term

Identify unmatched concepts by identifying the ones that don't join on the TERM_ID

In [ ]:
unmatched_concepts = subjects_concat_merge[subjects_concat_merge['TERM_ID'].isnull()][['color', 'subjects', 'media']]

In [ ]:
# subjects_concat_merge.loc[:,['media', 'subjects']]

In [ ]:
unmatched_concepts_by_color = unmatched_concepts.groupby(['color', 'subjects'])['media'].agg('count').reset_index()
unmatched_concepts_by_color.to_csv('csv_files/unmatched_concepts_by_color.csv')
# display(unmatched_concepts_by_color.sort_values('media', ascending=False))

In [ ]:
# review examples of results

sample = 'Inca'

df_combined_color_drop = df_combined_color[df_combined_color['join_concept'].notna()]
df_combined_color_drop = df_combined_color_drop[df_combined_color_drop['join_concept'].str.contains(sample)]
df_combined_search = df_combined_color_drop[
            ['media',
            'subject',
            'join_concept',
            'BoxId',
            'color']
]
df_combined_search


## Get translated Subjects from AAT Files

In [ ]:
empty_term = subjects_concat_merge[subjects_concat_merge['TERM_ID'].isnull()]
empty_term_n = empty_term[['subjects','media']].groupby(['subjects'])['media'] \
                                                .count() \
                                                .reset_index(name='count') \
                                                .sort_values(['count'], ascending=False)
empty_term_n

In [ ]:
alt_lang = (pd.merge(subjects_concat_merge, concepts, how='left', on='SUBJECT_ID_x'))
alt_lang = alt_lang[
    (alt_lang['PREFERRED_y_y'].isnull()) 
    | (alt_lang['PREFERRED_y_y'] == 'P')]

In [ ]:
espanol_code = '70641'

alt_lang_espanol = alt_lang[(alt_lang['LANGUAGE_CODE_y'] == espanol_code) 
                    | (alt_lang['LANGUAGE_CODE_y'].isnull())
                   ]
# alt_lang_espanol = alt_lang_espanol[alt_lang_espanol['media'] == 'A-2_Arquin_003_0349.jpg']
alt_lang_espanol[alt_lang_list]

# Below is junk... 

## Manipulate titles for translations

## Manipulate titles for translations

In [ ]:
# get the count of unique titles
count_of_titles = df_combined['title'].str.strip()
count_of_titles = df_combined['title'].unique()
count_of_titles = count_of_titles.tolist()
len(count_of_titles)

In [ ]:
# convert the list of unique titles to a dataframe and write to CSV so that they can be put into gSheet for translation
count_of_titles_df = pd.DataFrame(count_of_titles)
count_of_titles_df.to_csv('csv_files/titles.csv')

## Next Steps:

1. ~~Generate list of unique titles (for Tuesday morning, June 14, 2022)~~
1. Create an exceptions list for concepts containing commas (',') and reprocess accordingly
1. ~~See if I can extract table label color and group by - otherwise, group exceptions list by box number~~
1. Unmatched concepts list 
1. Unique subject list for translations

## Meanwhile for Ethan

1. Clean this up so that there is a load script
1. Define a notebook that cleanly generates the things required from above

## Where to work?

1. Spreadsheet: Data Design: https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1394255919



In [ ]:
df_combined_color

In [ ]:
df_combined_color = pd.merge(df_combined, color_match_df, on='BoxId')

In [ ]:
df_combined_color

Experiment with splitting subjects that contain commas (',') e.g. "serpent (general, animal)" from box 61

In [ ]:
df_combined_color_drop = df_combined_color[df_combined_color['join_concept'].notna()]
df_combined_color_drop = df_combined_color_drop[df_combined_color_drop['join_concept'].str.contains('Inca \(culture')]
df_combined_search = df_combined_color_drop[
            ['media',
            'subject',
            'join_concept',
            'BoxId',
            'color']
]
df_combined_search

In [ ]:
test_string = 'serpents (general, animals)'

In [ ]:
t = df_combined_search['subject'].to_list()[0]
print(t)

In [ ]:
test_string in t

In [ ]:
for x in concepts_comma:
    if x in t:
        print(x)

In [ ]:
media

In [ ]:


df_replace = df.loc[df['media'] == 'G-4_Arquin_051_0379.jpg']
x = df_replace.to_numpy()[0][3]
x = x.replace(test_string, test_string.replace(',', '∆'))
df.loc[df['media'] == 'G-4_Arquin_051_0379.jpg', 'subject'] = x

In [ ]:
df2 = df
df2.loc[df2["media"] == "G-4_Arquin_051_0379.jpg", "subject"] = x

In [ ]:
df = 

In [ ]:
for i, con in enumerate(concepts_comma):
    if df['subject'].str.contains(con).any():
        print(f'{i}: contains: {con}')

In [ ]:
# concepts = concepts['use_concept']
concepts_drop = concepts[concepts['use_concept'].notna()]

What's up with "presidents' houses"?

In [ ]:
subjects_concat_merge

In [ ]:
# df = df[df['Credit-Rating'].str.contains('Fair')]
concepts_drop = concepts[concepts['use_concept'].notna()]
concepts_drop = concepts_drop[concepts_drop['use_concept'].str.contains('presidents')]

In [ ]:
df_combined_color_drop = df_combined_color[df_combined_color['join_concept'].notna()]
df_combined_color_drop = df_combined_color_drop[df_combined_color_drop['join_concept'].str.contains('presidents')]
df_combined_search = df_combined_color_drop[
            ['media',
            'subject',
            'join_concept',
            'BoxId',
            'color']
]
df_combined_search.

## Step through the process for replacing commas - for troubleshooting

In [ ]:
df = return_box(51)

In [ ]:
search_term = 'Inca (culture, general)'
search_media = 'G-4_Arquin_051_0373.jpg'

In [ ]:
x = df.loc[df['media'] == search_media]

In [ ]:
# doesn't work for the example... 
y = check_for_commas(x)
y = split_df(y)
y

In [ ]:
df_media_subject = {}
df_media_subject = x.set_index('media').to_dict()['subject']
df_media_subject

In [ ]:
search_string = df_media_subject[search_media]
search_string

In [ ]:
search_term in concepts_comma

In [ ]:
find_string = False

comma_conc = search_term+','

re_search_term = search_term.replace('(', '\(')
re_search_term = re_search_term.replace(')', '\)')

if re_conc in search_string:
    find_string = True
if bool(re.search(f'{re_search_term}$', search_string)):
    find_string = True

find_string

In [ ]:
for i, media in enumerate(df_media_subject):
    if media == search_media:
        print(f'step 1: found media: {media})

    for j, conc in enumerate(concepts_comma):
        found_conc = False
        comma_conc = conc+','
    

    try:
        search_within = df_media_subject[media]
        re_conc = conc.replace('(', '\(')
        re_conc = re_conc.replace(')', '\)')

        if comma_conc in search_within:
            found_conc = True
        if bool(re.search(f'{re_conc}$', search_within)):
            found_conc = True

        if found_conc:
            now = datetime.now()
            current_time = now.strftime("%m-%d-%y %H:%M:%S")

            log_list.append([f'{current_time}: {i}: {j}: {media} contains subject: {conc}'])
#                 print(f'{i}: {j}: {media} contains subject: {conc}')
            df_replace = df.loc[df['media'] == media]
            try:
                x = df_replace.to_numpy()[0][3]
                x = x.replace(conc, conc.replace(',', '∆'))
                df.loc[df['media'] == media, 'subject'] = x
            except IndexError:
                print(f'Index Error, media = {media}, subject = {x}')
                break
    except TypeError:
        continue

Put the commas back

In [ ]:
after_x = df.loc[df['media'] == search_media]
after_x

In [ ]:
search_within = df_media_subject[media]
type(search_within)

In [ ]:
y = check_for_commas(x)
y = split_df(y)

In [ ]:
y

In [ ]:
z = y.loc[y['media'] == 'B-3_Arquin_010_0283.jpg']